In [ ]:
import time

from IPython.display import HTML

import numpy as np
import matplotlib.pyplot as plt
plt.rcdefaults()
import matplotlib.ticker as ticker
import torch
from tqdm import tqdm

from threedeequadsim import quadsim, controller, trajectory, experiments, utils

import importlib
importlib.reload(controller)
importlib.reload(quadsim)
importlib.reload(trajectory)

In [ ]:
nametag = 'hover'

In [ ]:
folder = './plots/' + time.strftime('%Y-%m-%d') + '/' + nametag + '/'
if not os.path.isdir(folder):
    os.makedirs(folder)
    print('Created data folder ' + folder)

In [ ]:
def savefig(plottag):
    plt.savefig(folder + plottag + '.png', bbox_inches='tight')
# def savefig(plottag):
#     pass
xyz = ['x', 'y', 'z']

In [ ]:
eta_a = 0.01

eta_A_convex = 0.001
eta_A_biconvex = 0.0002
eta_A_deep = 0.05

dim_a = 100
dim_A = 100
layer_sizes = (75, 150)

feature_freq = 0.5

In [ ]:
q_kwargs = {
    'Vwind' : 0.,  # mean wind speed
    'Vwind_cov' : 3.0,                # how quickly the wind changes
    'Vwind_gust' : 5.0,                # for hard wind constrant, wind speed is in the range Vwind +/- Vwind_gust
    'wind_constraint' : 'soft',
    't_stop' : 30.,
    'wind_update_period' : 2.0,
    'wind_model': 'iid'
}

In [ ]:
CTRLS = [
    # controller.Baseline(),
    controller.MetaAdaptBaseline(eta_a_base=eta_a, dim_a=dim_a, feature_freq=feature_freq),
    controller.MetaAdaptConvex(eta_a_base=eta_a, eta_A_base=eta_A_convex, dim_a=dim_a, dim_A=dim_A, feature_freq=feature_freq),
    controller.MetaAdaptBiconvex(eta_a_base=eta_a, eta_A_base=eta_A_biconvex, dim_a=dim_a, dim_A=dim_A, feature_freq=feature_freq),
    controller.MetaAdaptDeep(eta_a_base=eta_a, eta_A_base=eta_A_deep, dim_a=dim_a, layer_sizes=layer_sizes),
    controller.Omniscient()
]

In [ ]:
T = trajectory.hover

t_kwargs = {
    'pd' : np.zeros(3)
}

# T = trajectory.fig8
# t_kwargs = {
#     'T': 5.
# }

In [ ]:
Data = []

for c in CTRLS:
    q = quadsim.QuadrotorWithSideForce(**q_kwargs, )
    t = T(**t_kwargs)

    data = q.run(trajectory=t, controller=c, seed=116 ) # Note that this will set the seed, call c.reset_controller(), then reset the seed again
    Data.append(data)

In [ ]:
color = ((1,0,0), (0,1,0), (0,0,1)) # colors for x, y and z

In [ ]:
for j, data in enumerate(Data):
    print(CTRLS[j]._name)

    plt.figure(figsize=(5,3))

    # Plot position tracking
    plt.subplot(2,1,1)
    for i in [0, 2]: # only plot x and z
        # plt.subplot(2,3,i+1)
        plt.plot(data['t'], data['X'][:,i], label=xyz[i], color = color[i])
        plt.plot(data['t'], data['pd'][:,i], ':', label=xyz[i] + '_d', color = color[i])
        # plt.plot(data['t'], data['Vwind'][:,i], label='Vwind_' + xyz[i])
    plt.legend(bbox_to_anchor=(1.05, 1.))
    for t in data['t'][data['meta_adapt_trigger']]:
        plt.axvline(t, ls=':', color='k', lw=0.5)
    plt.xlabel('t')
    plt.ylabel('Position [m]')
    # plt.ylabel(xyz[i])
    # plt.ylim((-0.3, 0.3))

    # Plot force estimation
    plt.subplot(2,1,2)
    for i in [0, 2]:
        # plt.subplot(2,3,i+4)
        plt.plot(data['t'], data['Fs'][:,i], '--', label=r'$f_' + xyz[i] + '$', color = color[i])
        plt.plot(data['t'], data['f_hat'][:,0,i], label=r'$\hat{f}_' + xyz[i] + '$', color = color[i])
    plt.legend(bbox_to_anchor=(1.05, 1.1))
    for t in data['t'][data['meta_adapt_trigger']]:
        plt.axvline(t, ls=':', color='k', lw=0.5)
    plt.xlabel('t')
    plt.ylabel('Force [N]')
    # plt.ylabel(xyz[i])
    plt.ylim((-8,8))

    plt.suptitle(CTRLS[j].name_long)
    savefig('xz-performance_' + CTRLS[j]._name)

In [ ]:
print('%15s, %6s, %15s' % ('Controller', 'ACE', 'Steady State ACE'))

for c, data in zip(CTRLS, Data):
    err = experiments.get_error(data['X'], data['pd'])
    err_ss = experiments.get_error(data['X'], data['pd'], istart=1000) # ss=steady state
    print('%15s, %5.4f, %5.4f' % (c._name, err['meanerr'], err_ss['meanerr']))

In [ ]:
import rowan
import collections
import os
import shutil

In [ ]:
Circle = collections.namedtuple('Circle', 'center radius normal ax1 ax2', defaults=((0., 0., 0.), (0., 0., 0.)))
l_arm = 0.3
D = 0.45 # params['D'] * 3
h = 0.10 # params['h']
lines = (((0.,0.,0.),(l_arm,l_arm,0.)),
                ((0.,0.,0.),(-l_arm,l_arm,0.)),
                ((0.,0.,0.),(-l_arm,-l_arm,0.)),
                ((0.,0.,0.),(l_arm,-l_arm,0.)),
                ((l_arm,l_arm,0.),(l_arm,l_arm,h)),
                ((-l_arm,l_arm,0.),(-l_arm,l_arm,h)),
                ((-l_arm,-l_arm,0.),(-l_arm,-l_arm,h)),
                ((l_arm,-l_arm,0.),(l_arm,-l_arm,h)),
                )
# Make circles mutable so that we can update the ax1 and ax2 fields
circles = [Circle((l_arm, l_arm, h), D/2, (0., 0., 1.)),
                Circle((-l_arm, l_arm, h), D/2, (0., 0., 1.)),
                Circle((-l_arm, -l_arm, h), D/2, (0., 0., 1.)),
                Circle((l_arm, -l_arm, h), D/2, (0., 0., 1.)),]

for i, (center, radius, normal, ax1, ax2) in enumerate(circles):
    ax1 = np.cross(normal, (1., 0., 0.))
    if np.linalg.norm(ax1) < 1e-3:
        ax1 = np.cross(normal, (0., 0., 1.))
    ax1 = ax1 / np.linalg.norm(ax1)
    ax2 = np.cross(normal, ax1)
    circles[i] = Circle(np.array(center), radius, np.array(normal), np.array(ax1), np.array(ax2))

In [ ]:
def plot_quad(ax, R, p):
    for line in lines:
        x, y, z = R @ np.array(line).transpose() + p[:, np.newaxis]
        ax.plot3D(x,y,z, 'k', lw=3)

    for (center, radius, normal, ax1, ax2) in circles:
        center = R @ center
        ax1 = R @ ax1
        ax2 = R @ ax2
        theta = np.linspace(0, 2*np.pi)
        x, y, z = radius * np.outer(ax1, np.cos(theta)) + radius * np.outer(ax2, np.sin(theta)) + center[:, np.newaxis] + p[:, np.newaxis]
        ax.plot3D(x,y,z, 'k', lw=3)


In [ ]:
fps = 0.05

data = Data[0]
j = 0
for j, data in enumerate(Data):
    t_next_frame = 0.
    subfolder = folder + '/' + CTRLS[j]._name + '_gif-frames'
    if os.path.isdir(subfolder):
        print('Warning: overwriting dataset in folder' + subfolder)
        # os.rmdir(folder)
        shutil.rmtree(subfolder)
    os.makedirs(subfolder)
    # if not os.path.isdir(subfolder):
    #     os.makedirs(subfolder)
    print('Created data folder ' + subfolder)

    def saveframe(frameid = [0]):
        plt.savefig(subfolder + '/%04d' % frameid[0], bbox_inches='tight')
        frameid[0] += 1

    xs, ys, zs = np.array(data['X'][:, 0:3]).transpose()
    xmid = (min(xs) + max(xs))/2
    ymid = (min(ys) + max(ys))/2
    zmid = (min(zs) + max(zs))/2
    xwid = max(xs) - min(xs)
    ywid = max(ys) - min(ys)
    zwid = max(zs) - min(zs)
    margin = 0.5
    wid = max((xwid, ywid, zwid)) + 2*margin

    for k, t in enumerate(data['t']):
        if t >= t_next_frame:
            t_next_frame += 1/fps

            fig = plt.figure()
            # gs = fig.add_gridspec(2,2)
            gs = plt.GridSpec(2, 2)

            # Plot quadrotor
            ax = fig.add_subplot(gs[:,0], projection='3d')
            # ax = plt.axes(projection='3d')
            p = data['X'][k,:3]
            q = data['X'][k, 3:7]
            R = rowan.to_matrix(q)
            plot_quad(ax, R, p)
            ax.set_xlim(xmid - wid/2, xmid + wid/2)
            ax.set_ylim(ymid - wid/2, ymid + wid/2)
            ax.set_zlim(zmid - wid/2, zmid + wid/2)
            vwind = data['Vwind'][k]
            plt.quiver(xmid - wid/2, ymid, zmid + wid/2, vwind[0], vwind[1], vwind[2])
            ax.text(xmid - wid/2, ymid - wid/2, zmid - wid/2, 't=%.2fs' % t)
            

            # Plot position tracking
            plt.subplot(gs[0,1])
            for i in [0, 2]: # only plot x and z
                plt.plot(data['t'][:k], data['X'][:k,i], label=xyz[i], color = color[i])
                plt.plot(data['t'][:k], data['pd'][:k,i], ':', label=xyz[i] + '_d', color = color[i])
            plt.legend(bbox_to_anchor=(1.05, 1.))
            for t in data['t'][data['meta_adapt_trigger']]:
                plt.axvline(t, ls=':', color='k', lw=0.5)
            plt.xlabel('t')
            plt.ylabel('Position [m]')
            plt.ylim((-0.3, 0.3))

            # Plot force estimation
            plt.subplot(gs[1,1])
            for i in [0, 2]:
                plt.plot(data['t'][:k], data['Fs'][:k,i], '--', label=r'$f_' + xyz[i] + '$', color = color[i])
                plt.plot(data['t'][:k], data['f_hat'][:k,0,i], label=r'$\hat{f}_' + xyz[i] + '$', color = color[i])
            plt.legend(bbox_to_anchor=(1.05, 1.1))
            for t in data['t'][data['meta_adapt_trigger']]:
                plt.axvline(t, ls=':', color='k', lw=0.5)
            plt.xlabel('t')
            plt.ylabel('Force [N]')
            plt.ylim((-8,8))

            plt.suptitle(CTRLS[j].name_long)
            plt.tight_layout()
            saveframe()
            plt.close()
            # plt.show()

In [ ]:
vwind